In [1]:
from datetime import datetime

In [1]:
import psycopg2
import pandas as pd
import datacompy
import pyodbc
import useful_functions as use

In [2]:
conn = psycopg2.connect(
    host='oneview-dev-content-aurora-cluster.cluster-cwp4vd8mllvz.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='m3CUmKz0OEOA0Mej')

In [3]:
username = 'vigrose'
password = 'Ravenclaw~10946'
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [3]:
#OneView orders query
ov_orders_sql = '''
    SELECT * FROM ONEVIEW.CREDENTIALING_ORDER
    '''
ov_orders = pd.read_sql_query(ov_orders_sql, conn)

In [5]:
ppd = use.get_ppd()

INFO:useful_functions:PPD file from Jan 08 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
ppd[ppd.ME=='02312071625'][['LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD']]

,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,MAILING_LINE_2,CITY,STATE,ZIP,MD_DO_CODE,...,BIRTH_COUNTRY,GENDER,TELEPHONE_NUMBER,PRESUMED_DEAD_FLAG,FAX_NUMBER,TOP_CD,PE_CD,PRIM_SPEC_CD,SEC_SPEC_CD,MPA_CD
343438,WRIGHT,ELIZABETH,SUZANNE,NaN,673RD MEDICAL GROUP,5955 ZEAMER AVE,JBER,AK,99506,1,...,US1,2.0,3.178524e+09,NaN,3.178521e+09,20,30,FM,US,OFF


In [4]:
ov_orders

,id,customer,product,number,medical_education_number,date,quantity,unique_physician_identification_number
0,3214298,146574,4915491,43962242,02312071625,2021-01-01,1,None
1,3214299,127592,4915513,43962236,01720130579,2021-01-01,1,None
2,3214300,127592,4915513,43962236,03515920012,2021-01-01,1,G33831
3,3214301,127592,4915513,43962236,49666020021,2021-01-01,1,None
4,3214302,127592,4915513,43962236,57502080204,2021-01-01,1,None
...,...,...,...,...,...,...,...,...
3638429,3804567,58844,4915514,45065979,01902930201,2022-01-08,1,H25608
3638430,3804568,52008,4915513,45071288,01001961612,2022-01-10,1,I35196
3638431,3804569,52008,4915513,45071220,02501041028,2022-01-10,1,None
3638432,3804570,52008,4915513,45071054,03601871351,2022-01-10,1,E85151


In [ ]:
#OneView orders query
ov_ppd_sql = '''
    SELECT * FROM ONEVIEW.PHYSICIAN
    '''
ov_ppd = pd.read_sql_query(ov_ppd_sql, conn)

In [ ]:
#OneView CUSTOMER query
ov_customer_sql = '''
    SELECT * FROM ONEVIEW.CREDENTIALING_CUSTOMER
    '''
ov_customer = pd.read_sql_query(ov_customer_sql, conn)

In [ ]:
f"""
            SELECT DISTINCT
            D.FULL_DT,
            H.MED_EDU_NBR AS ME,
            H.PARTY_ID,
            O.ORDER_NBR,
            O.ORDER_PRODUCT_ID,
            O.ORDER_PHYSICIAN_HIST_KEY,
            O.CUSTOMER_KEY
            FROM
            AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
            WHERE
            D.DATE_KEY = O.ORDER_DT_KEY
            AND
            H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
            AND
            D.YR in {years}
            AND
            O.ORDER_PRODUCT_ID IN {products}
            """

In [ ]:
len(dm_orders_2)

In [ ]:
#DataMart orders query
dm_orders_sql = '''
    SELECT
    O.FACT_EPROFILE_KEY AS ID,
    O.CUSTOMER_KEY AS CUSTOMER,
    O.ORDER_PRODUCT_ID AS PRODUCT, 
    O.ORDER_NBR AS NUMBER,
    H.MED_EDU_NBR AS MEDICAL_EDUCATION_NUMBER,
    D.FULL_DT AS DATE,
    O.QUANTITY,
    H.UPIN_NBR AS UNIQUE_PHYSICIAN_IDENTIFICATION_NUMBER
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    '''
dm_orders_2 = pd.read_sql(con=AMADM, sql=dm_orders_sql)

In [7]:
datetime.today().month

11

In [17]:
#DataMart orders query
dm_orders_sql = f'''
    SELECT DISTINCT
    O.FACT_EPROFILE_KEY AS ID,
    O.CUSTOMER_KEY AS CUSTOMER,
    O.ORDER_PRODUCT_ID AS PRODUCT, 
    O.ORDER_NBR AS NUMBER,
    H.MED_EDU_NBR AS MEDICAL_EDUCATION_NUMBER,
    D.FULL_DT AS DATE,
    O.QUANTITY,
    H.UPIN_NBR AS UNIQUE_PHYSICIAN_IDENTIFICATION_NUMBER
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND 
    D.MONTH_NBR = {datetime.today().month}
    ORDER BY FACT_EPROFILE_KEY
    '''
dm_orders_3 = pd.read_sql(con=AMADM, sql=dm_orders_sql)

In [16]:
dm_orders_3

,ID,CUSTOMER,PRODUCT,NUMBER,MEDICAL_EDUCATION_NUMBER,DATE,QUANTITY,UNIQUE_PHYSICIAN_IDENTIFICATION_NUMBER
0,3390355,53389,4922511,44293730,04113031714,2021-04-17,1,None
1,3609621,53389,4915503,44715117,-1,2021-08-24,1,None
2,3609622,53389,4915503,44715117,-1,2021-08-24,1,None
3,3609623,53389,4915503,44715117,-1,2021-08-24,1,None
4,3609624,53389,4915503,44715117,-1,2021-08-24,1,None
...,...,...,...,...,...,...,...,...
520855,3735252,154113,4915514,44913825,01720001493,2021-11-15,1,I07719
520856,3735253,154113,4915514,44912926,67202120111,2021-11-15,1,None
520857,3735254,52666,4915513,44913591,04813190597,2021-11-15,1,None
520858,3735255,59786,4915513,44912733,70402020596,2021-11-15,1,None


In [ ]:
#DataMart orders query
dm_orders_sql = '''
    SELECT 
    O.FACT_EPROFILE_KEY AS ID,
    O.CUSTOMER_KEY AS CUSTOMER,
    O.ORDER_PRODUCT_ID AS PRODUCT, 
    O.ORDER_NBR AS NUMBER,
    H.MED_EDU_NBR AS MEDICAL_EDUCATION_NUMBER,
    D.FULL_DT AS DATE,
    O.QUANTITY,
    H.UPIN_NBR AS UNIQUE_PHYSICIAN_IDENTIFICATION_NUMBER
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND 
    D.FULL_DT>'2021-11-11'
    '''
dm_orders_3 = pd.read_sql(con=AMADM, sql=dm_orders_sql)

In [ ]:
dm_orders_1.drop_duplicates('ID')

In [ ]:
dm_orders_1.columns = [c.lower() for c in dm_orders_1.columns.values]

In [ ]:
dm_orders = dm_orders_1[dm_orders_1.medical_education_number.isin(ov_ppd.medical_education_number)]

In [ ]:
ov_orders['unique_physician_identification_number'] = ['None' if x=='(null)' else x for x in ov_orders.unique_physician_identification_number]

In [ ]:
ov_orders['date']=pd.to_datetime(ov_orders.date)

In [ ]:
dm_orders['date']=pd.to_datetime(dm_orders.date)

In [ ]:
order_compare = datacompy.Compare(ov_orders, dm_orders, join_columns='id', df1_name = 'oneview', df2_name = 'datamart')

In [ ]:
print(order_compare)

In [ ]:
dm_orders['customer']=dm_orders.customer.astype(int)

In [ ]:
test_2 = dm_orders[dm_orders.customer.isin(ov_customer.id)]

In [ ]:
test_2[test_2.medical_education_number.isin(ov_ppd.medical_education_number)]

In [ ]:
test_2[test_2.duplicated(subset='medical_education_number', keep=False)]

In [ ]:
len(test_2)

In [ ]:
#1,650,509
#1,651,281

In [ ]:
1901349/3552630

In [ ]:
print(order_compare.report())

In [ ]:
order_test = order_compare.df2_unq_rows

In [ ]:
order_test

In [ ]:
order_compare.df2_unq_rows.sample(100).to_csv('../../Data/OneView_Testing/Missing_CredentialingOrder_Rows_211102.csv', index=False)

In [ ]:
ov_orders[ov_orders.customer.isin(ov_customer.id)]

In [ ]:
len(ov_customer)

In [ ]:
#OneView Product query
ov_product_sql = '''
    SELECT * FROM ONEVIEW.CREDENTIALING_PRODUCT
    '''
ov_product = pd.read_sql_query(ov_product_sql, conn)

In [ ]:
len(ov_product)

In [ ]:
#DataMart Product query
dm_product_sql = '''
    SELECT
    PRODUCT_ID AS ID,
    PRODUCT_DESC AS DESCRIPTION
    FROM 
    AMADM.DIM_PRODUCT
    '''
dm_product = pd.read_sql(con=AMADM, sql=dm_product_sql)

In [ ]:
product_4 = pd.read_csv('../../Data/MasterfileCore/credentialing_product_new.csv')

In [ ]:
product_4 = product_4.rename(columns={'product_id':'id','product_desc':'description'})

In [ ]:
product_4.columns

In [ ]:
product_compare = datacompy.Compare(dm_product, ov_product, join_columns='id', df1_name = 'datamart', df2_name = 'udrive')

In [ ]:
print(product_compare.report())

In [ ]:
product_compare.sample_mismatch(column="description", sample_count=315).dropna().rename(columns={'description_df1':'description_oneview','description_df2':'description_datamart'})

In [ ]:
product_compare.df2_unq_rows.sample(100).to_csv('../../Data/OneView_Testing/Missing_CredentialingProduct_Rows.csv', index=False)

In [ ]:
pd.read

In [ ]:
#DataMart Product query
dm_customer_sql = '''
    SELECT DISTINCT
    CUSTOMER_KEY AS id,
    CUSTOMER_NBR AS number,
    CUSTOMER_NAME as name,
    CUSTOMER_TYPE as type,
    CUSTOMER_TYPE_DESC as type_description,
    CUSTOMER_CATEGORY as category,
    CUSTOMER_CATEGORY_DESC as category_description,
    CURRENT_IND as current_indicator,
    DW_UPDATED_TS
    FROM 
    AMADM.DIM_CUSTOMER
    '''
dm_customer = pd.read_sql(con=AMADM, sql=dm_customer_sql)

In [ ]:
dm_customer

In [ ]:
org_addresses = pd.read_excel('../../Data/MasterfileCore/Org_Addresses.xlsx')

In [ ]:
org_addresses = org_addresses.rename(columns={'street_one':'address_1','street_two':'address_2','street_three':'address_3'})

In [ ]:
dm_customer.columns = [c.lower() for c in dm_customer.columns.values]

In [ ]:
dm_customer['number'] = [str(int(x)) for x in dm_customer.number]
org_addresses['number'] = [str(x) for x in org_addresses.number]

In [ ]:
dm_customer_2 = pd.merge(org_addresses, dm_customer, on='number', how='right')

In [ ]:
dm_customer['customer_id']=dm_customer.number.astype('int64')

In [ ]:
dm_customer[dm_customer.customer_id.isin(ov_orders.customer)]

In [ ]:
customer_compare = datacompy.Compare(ov_customer, dm_customer_2, join_columns='id', df1_name = 'oneview', df2_name = 'datamart')

In [ ]:
print(customer_compare.report())

In [ ]:
customer_compare.df2_unq_rows.to_csv('../../Data/OneView_Testing/Missing_CredentialingCustomer_Rows_211102.csv', index=False)

In [ ]:
customer_compare.df2_unq_rows.sample(100).to_csv('../../Data/OneView_Testing/Missing_CredentialingCustomer_Rows.csv', index=False)

In [ ]:
cust = customer_compare.df2_unq_rows

In [ ]:
dm_customer_2['updated'] = pd.to_datetime(dm_customer_2.dw_updated_ts)

In [ ]:
dm_customer_2[(dm_customer_2.updated>datetime(2021,10,13))&(dm_customer_2.id.isin(misindicator.id))]

In [ ]:
cust[cust.updated<datetime(2021,10,13)]

In [ ]:
name_test = customer_compare.sample_mismatch(column="name", sample_count=683)

In [ ]:
dm_customer[dm_customer.id==158351]

In [ ]:
misindicator = customer_compare.sample_mismatch(column="current_indicator", sample_count=73)

In [ ]:
customer_compare.sample_mismatch(column="current_indicator", sample_count=73).to_csv('../../Data/OneView_Testing/CredentialingCustomer_Current_Indicator_mismatches_update.csv', index=False)

In [ ]:
ov_customer.columns

In [ ]:
dm_customer.columns

In [ ]:
name_test.dropna(subset=['name_df2'])

In [ ]:
duplicates = pd.read_csv('../../Data/MasterfileCore/duplicates.csv')

In [ ]:
duplicates = duplicates[dm_orders.columns]

In [ ]:
dm_orders.sort_values('id')

In [ ]:
duplicates[~duplicates.id.isin(dm_orders.id)]

In [ ]:
#NOT THIS ONE
#DataMart orders query
dm_orders_sql = '''
    SELECT
    O.FACT_EPROFILE_KEY AS ID,
    O.CUSTOMER_KEY AS CUSTOMER,
    O.ORDER_PRODUCT_ID AS PRODUCT, 
    O.ORDER_NBR AS NUMBER,
    H.MED_EDU_NBR AS MEDICAL_EDUCATION_NUMBER,
    D.FULL_DT AS DATE,
    O.QUANTITY,
    H.UPIN_NBR AS UNIQUE_PHYSICIAN_IDENTIFICATION_NUMBER
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    '''
dm_orders_1 = pd.read_sql(con=AMADM, sql=dm_orders_sql)

In [ ]:
ov_ppd[ov_ppd.first_name=="DE'ANDRE"]

In [1]:
ov_ppd

NameError: name 'ov_ppd' is not defined